# Homework 03

This model is based on the one we have been examining in class. 

$$
v(k) = \max_{f(k) \geq k' \geq 0} u(f(k) - k') + \beta v(k')
$$

# Instructions

## Part 1, due Monday

1. Derive the analytic values for the value and policy functions when $f(k) = Ak^\alpha$ instead of $f(k) = k^\alpha$
2. Suppose that we have a finite-horizon model that ends in period $t=T$. What must $k_{T+1}$ be? Why?
3. In the `SimpleGrowthModel` below, I have implemented the following production function: $f(k) = k^\alpha$
    
    Now create a new type, `GrowthModel <: AbstractDeterministicGrowthModel` that implements allowing a new "technology" parameter
    $$
    f(k) = Ak^\alpha
    $$
4. Create functions to ouptut utility, production, kstar, cstar, and vstar
5. Start with initial guess $V^T=0$. Now compute and plot $V^{T-i} \gets (Tv^{T-(i-1)})(k)$ for $i \in \{0,1,2,3,\ldots,10,15,20,25,50,100,200,300,400,500 \}$ and plot. What happens as $i$ increases?
6. Try this for  $\beta \in \{0.0, 0.01, 0.1, 0.5, 0.9, 0.99, 1.0\}$. What's different as $\beta$ changes?
7. Plot the optimal policy function $k' = g(k)$ for $\beta \in \{0.0, 0.01, 0.1, 0.5, 0.9, 0.99, 1.0\}$, along with the 45 degree line. What changes as $\beta$ changes? Why?

## Part 2, due Wednesday

Suppose that agents can choose an action $y \in \{0,1\}$. The payoffs to each are
\begin{align*}
u_0(X,\epsilon) &= \epsilon_{i0} \\
u_1(X,\epsilon) &= x_i^\top \beta + \epsilon_{i1}
\end{align*}
The shocks $\epsilon_{i0},\epsilon_{i1}$ are distributed as iid Type-I extreme value with mean 0 and scale parameter 1. Agents choose $y_i=1$ if $u_1 \geq u_0$ and $y_0$ if $u_1 < u_0$

I have simulated $X$ and $y$ below. 

1. Write down the log likelihood function for each individual $i$, $\log L_i(y_i|X_i)$ and the score $\nabla_\beta \log L_i(y_i|X_i)$. Feel free to consult Greene
2. Write a function to compute the log likelihood of the data: $\sum_i \log L_i(y_i|X_i)$ and the gradient of this function.
3. Use the finite difference capabilities of `Calculus.jl` to check your gradient and make sure that it's correct
4. Maximize the likelihood and compute standard errors using the Information Matrix. These should be

    $$
    \left[\sum_i \nabla \log L_i \nabla \log L_i^\top\right]^{-1} \to Var(\beta)
    $$
    
    Greene ch 21 (5th ed) has the appropriate formulas
    
You might be able to make good use of the following functions from [`StatsFuns.jl`](https://github.com/JuliaStats/StatsFuns.jl)

```julia
logsumexp      # log(exp(x) + exp(y)) or log(sum(exp(x)))
softmax        # exp(x_i) / sum(exp(x)), for i
```

In [ ]:
using Plots
gr(fmt = :png);

### Define a model type

In [ ]:
# create a heirarchy of abstract types in case we want to
# recycle functions
abstract type AbstractDeterministicGrowthModel end

# create a new "type"
struct SimpleGrowthModel <: AbstractDeterministicGrowthModel
    α::Float64  # capital share
    β::Float64  # discount rate
end

struct GrowthModel <: AbstractDeterministicGrowthModel
   # fill in parameters!! 
end

# when we create a type, it's best to access fields via functions
_α(x::AbstractDeterministicGrowthModel) = x.α
_β(x::AbstractDeterministicGrowthModel) = x.β
_αβ(g::AbstractDeterministicGrowthModel) = _α(g) * _β(g)

_A(g::GrowthModel) = 0.0 # FIXME

# create a default constructor in case we don't supply arguments
SimpleGrowthModel() = SimpleGrowthModel(0.65, 0.95);

In [ ]:
# create new instance of a model
SimpleGrowthModel(0.5, 0.8)

### Construct payoffs and production functions

In [ ]:
# you'll need to make a new production function
production(model::SimpleGrowthModel, k) = k^_α(model)

# but you won't need to change these
consumption(model::AbstractDeterministicGrowthModel, k, kp) = production(model, k) - kp
utility(    model::AbstractDeterministicGrowthModel, k, kp) = log(max(consumption(model, k, kp), 0))

### Create analytic solution

In [ ]:
kpstar(model::SimpleGrowthModel,k) = _αβ(model) * production(model,k)

cstar(model::SimpleGrowthModel,k) = (1-_αβ(model)) * production(model,k)

function vfstar(model::SimpleGrowthModel, k)
    ab = _αβ(model)
    A = (log(1 - ab) + log(ab) * ab / (1 - ab)) / (1 - _β(model))
    B = _α(model) / (1 - ab)
    return A + B * log(k)
end

### Plot analytidc solution

In [ ]:
# number of points in state-space grid
numk = 50

# state space
k_space = range(0.01, 0.5, length=numk)

g = SimpleGrowthModel()

In [ ]:
vftrue = broadcast(k -> vfstar(g,k), k_space)
kptrue = broadcast(k -> kpstar(g,k), k_space);

In [ ]:
# plot true policy function
plot_pftrue = plot(
    k_space, hcat(k_space, kptrue);
    labels = ["\$k'=k\$", "\$k^{'*}(k)\$"],
    xlabel="\$k_t\$",
    ylabel="\$k_{t+1}\$",
    title="Policy function",
    legend=false
)

# plot true value function
plot_vftrue = plot(
    k_space, vftrue;
    title="Value function",
    xlabel = "\$k\$",
    legend=false
)

plot(plot_pftrue, plot_vftrue)

# Define the Bellman operator

$$
(Tv)(k) = \max_{k' \in \Gamma(k)} \{u(k,k') + \beta v(k')  \}
$$

In [ ]:
# bellman operator with in-place updating
function bellman!(vf1, pf1, model::AbstractDeterministicGrowthModel, vf0, state_space)

    # defensive programming!
    size(vf1) == size(pf1) == size(vf0) || throw(DimensionMismatch())
    length(state_space) == length(vf1) || throw(DimensionMismatch())

    # for each possible state today
    for (i,k) in enumerate(state_space)
               
        # given the state, check each possible action and pick the optimal one
        # in general, we'll need to be more careful about how actions map to states in t+1
        kp_space = state_space
        
        # payoff today (return) given choice of kprime
        f(kp) = utility(model, k, kp)

        # pick the best kp given the static + dynamic payoff
        vf1[i], pf1[i] = findmax(f.(kp_space) .+ _β(g) .* vf0)
    end
end


# without in-place updating
function bellman(model, vf0, state_space)
    vf1 = similar(vf0)
    pf1 = Array{Int}(undef, size(vf0))
    bellman!(vf1, pf1, model, vf0, state_space)
end

## Work with Bellman operator

In [ ]:
vf0 = zeros(numk)
vf1 = similar(vf0)
pf = Vector{Int}(undef, numk)

pltv = plot(k_space, vf0, color = :black, linewidth = 2, alpha = 0.8, lab="")

# create new instance of a model
g = SimpleGrowthModel()

maxit = 5
for i in 1:maxit
    # Do one iteration
    bellman!(vf1, pf, g, vf0, k_space)
    
    # update vf1
    vf0 .= vf1
    
    # plot iteration
    plot!(pltv, k_space, vf1, color = RGBA(i/maxit, 0, 1 - i/maxit, 0.8), linewidth = 2, alpha = 0.6, lab="")
    
end

plot!(pltv, k_space, (x)->0.0, color = :black, linewidth = 2, alpha = 0.8, lab="\$V^T\$")
plot!(pltv, k_space, vftrue, color = :black, linewidth = 2, alpha = 0.8, lab = "\$\\hat V = V^{T-\\infty}\$")
plot!(pltv, legend = :right)

# Part 2

In [ ]:
using Random
using StatsFuns
using Optim
using LinearAlgebra
using Distributions

In [ ]:
Random.seed!(1234)

nobs = 100
β = [1.0, -2.0, 1.0, 2.0]
k = length(β)
X = randn(nobs, k)

# choice utilities
u0 = zeros(nobs)
u1 = X*β
u = hcat(u0, u1)

# multinomial logit probabilities
prob_actions = mapslices(softmax, u; dims=2)
cum_prob = cumsum(prob_actions; dims=2)
@assert all(cum_prob[:,2] .≈ 1)

# instead of simulating random type-1 extreme values, we just
# use a uniform variable and the CDF
e_quantile = rand(nobs)
y = [searchsortedfirst(view(cum_prob, i, :), q) for (i,q) in enumerate(e_quantile) ] .- 1 ;

### Your turn!

You can probably just fill these functions in. :)

In [ ]:
function checksizes(y,X,theta)
    n,k = size(X)
    n == length(y) || throw(DimensionMismatch())
    k == length(theta) || throw(DimensionMismatch())
    return n, k
end

function loglik(y::AbstractVector{Int}, X::AbstractMatrix, theta::AbstractVector)
    n,k = checksizes(y,X,theta)

    # compute log likelihood
    
    return -LL  # I *think* you'll need to flip sign to maximize
end


function dloglik!(grad::AbstractVector, y::AbstractVector{Int}, X::AbstractMatrix, theta::AbstractVector)
    n,k = checksizes(y,X,theta)    
    k == length(grad) || throw(DimensionMismatch())
    
    # compute gradient
    
    grad .*= -1  # flip signs to 
    return grad
end

dloglik(y::AbstractVector{Int}, X::AbstractMatrix, theta::AbstractVector) = dloglik!(similar(theta), y, X, theta)

function informationmatrix(y::AbstractVector{Int}, X::AbstractMatrix, theta::AbstractVector)
    n,k = checksizes(y,X,theta)    
    infomatrix = zeros(k,k)
    for i in 1:n
       infomatrix .+= 0 # FIXME
    end
    return -infomatrix # maybe flip signs?
end

In [ ]:
# define some wrappers
theta0 = zeros(k)

f(thet) = loglik(y,X,thet)

g!(grad,thet) = dloglik!(grad,y,X,thet)


# Check your gradients
@assert Calculus.gradient(f, theta0) ≈ dloglik(y,X,theta)

In [ ]:
@show results = optimize(f, g!, theta0, BFGS(); show_trace=true)

In [ ]:
theta1 = results.minimizer  # should be about β
Vcovinv = informationmatrix(y, X, theta1)
stderr = sqrt(diag(inv(Vcovinv)))
pvalues = 0.0 # your turn to figure out this